In [5]:
from twitter_scraper import settings
from twitter_scraper import utils
from twitter_scraper.clean import TWEET_DTYPE

import datetime as dt
import pandas as pd

START_DATE = dt.datetime.fromisoformat('2022-10-01T00:00:00+00:00')
END_DATE = dt.datetime.fromisoformat('2022-10-31T23:59:59+00:00')

tweets_dfs = utils.read_directory_files(settings.CLEAN_TWEETS_DIR, pd.read_csv, parse_dates=['created_at'], dtype=TWEET_DTYPE)
tweets_df = pd.concat(tweets_dfs)
tweets_df['user_id_str'] = tweets_df['user_id'].map(str)
# tweets_view = pd.read_csv('preproc_tweets_view.csv', parse_dates=['created_at'])

tweets_df = tweets_df[
    (tweets_df.created_at >= START_DATE)
    & (tweets_df.created_at <= END_DATE)
]

# tweets_view = tweets_view[
#     (tweets_view.created_at >= START_DATE)
#     & (tweets_view.created_at <= END_DATE)
# ]

In [6]:
ukraine_df = tweets_df[
    (tweets_df.apply(lambda row: any(
        kwd in row.full_text.lower() 
        for kwd in settings.KEYWORDS['ukraine_war']['include']
    ), axis=1))
]
# ukraine_df = ukraine_df[
#     ~ukraine_df.apply(lambda row: any(
#         kwd in row.full_text.lower() 
#         for kwd in kwds['exclude']
#     ), axis=1)
# ]

In [7]:
ukraine_df[list(set(ukraine_df.columns).difference(['year','quarter','quarter_name','month','month_name','week','week_name','day','day_name','hour','minute','second']))
].to_csv('adhoc_tweets_202210.csv', index=False, encoding='utf-8')


In [55]:
ukraine10_user_ids = ukraine_df.user_id.unique()

In [49]:
# ukraine11_user_ids = ukraine_df.user_id.unique()

In [57]:
print('10:', len(ukraine10_user_ids))
print('10:', len(ukraine11_user_ids))

unique_users = set(list(ukraine10_user_ids) + list(ukraine11_user_ids))
print('unique:', len(unique_users))

10: 2257
10: 2238
unique: 3029


In [60]:
from twitter_scraper.clean import USER_DTYPE
users_dfs = utils.read_directory_files(settings.CLEAN_USERS_DIR, pd.read_csv, parse_dates=['created_at'], dtype=USER_DTYPE)
users_df = pd.concat(users_dfs)

In [62]:
users_df[users_df.user_id.isin(unique_users)].to_csv('adhoc_users.csv', index=False, encoding='utf-8')